In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [2]:
with open('/Users/aadityajoshi/Downloads/therapy_intents.json','r') as f:
    intents = json.load(f)

In [3]:
uni_tags = []
tags = []
patterns = []
responses = []
for intent in intents['intents']:
    tag = intent['tag']
    uni_tags.append(tag)
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(tag)
        
    

In [4]:
df = pd.DataFrame(patterns,columns=['patterns'])

In [5]:
df['tags'] = pd.DataFrame(tags)

In [6]:
df

,patterns,tags
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hi there,greeting
4,Hello,greeting
...,...,...
227,How do I know if I'm unwell?,fact-29
228,How can I maintain social connections? What if...,fact-30
229,What's the difference between anxiety and stress?,fact-31
230,What's the difference between sadness and depr...,fact-32


In [7]:
tg = list(df['tags'].unique())

In [8]:
X = df['patterns']
y = df['tags']

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
clf = Pipeline([('vect',TfidfVectorizer()),
               ('rfc',RandomForestClassifier())])
clf.fit(X,y)

Pipeline(steps=[('vect', TfidfVectorizer()), ('rfc', RandomForestClassifier())])

In [10]:
clf.predict(['how much time to receive?'])[0]

'delivery'

In [9]:
import random
res_dict = {}
for intent in intents['intents']:
    tag = intent['tag']
    response = intent['responses']
    res_dict[tag] = response

In [12]:
def chat_bot():
    run = True
    print("Hello Sharpaxis here how may I help you?\n")
    while run:
        message = input("You: ")
        if message.strip().lower() == 'quit':
            run = False
            break
        pred = clf.predict([message])[0]
        bot_response = random.choice(res_dict[pred])
        print(bot_response)

In [13]:
chat_bot()

Hello Sharpaxis here how may I help you?

You: quit


In [10]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [11]:
def lemmatize(text):
    doc = nlp(text.lower())
    for token in doc:
        return token.lemma_

In [12]:
df['patterns'] = df['patterns'].apply(lemmatize)

In [13]:
blanks = []
for i,rv,lb in df.itertuples():
    if type(rv) == str:
        if rv.isspace():
            blanks.append(i)
blanks

[]

In [14]:
df.dropna(inplace=True)

In [15]:
def vect(text):
    doc = nlp(text)
    return doc.vector

In [16]:
df['vect'] = df['patterns'].apply(vect)

In [17]:
X_train = df['vect'].values
y_train = df['tags'].values

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(1280, activation='relu', input_shape=(300,)))
model.add(Dropout(0.5))
model.add(Dense(640, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(320, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy')


In [20]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [21]:
tg
tg_map = {}
for tag,i in enumerate(tg):
    if tag not in tg_map:
        tg_map[i] = tag
        

In [22]:
tg_map

{'greeting': 0,
 'morning': 1,
 'afternoon': 2,
 'evening': 3,
 'night': 4,
 'goodbye': 5,
 'thanks': 6,
 'no-response': 7,
 'neutral-response': 8,
 'about': 9,
 'skill': 10,
 'creation': 11,
 'name': 12,
 'help': 13,
 'sad': 14,
 'stressed': 15,
 'worthless': 16,
 'depressed': 17,
 'happy': 18,
 'casual': 19,
 'anxious': 20,
 'not-talking': 21,
 'sleep': 22,
 'scared': 23,
 'death': 24,
 'understand': 25,
 'done': 26,
 'suicide': 27,
 'hate-you': 28,
 'hate-me': 29,
 'default': 30,
 'jokes': 31,
 'repeat': 32,
 'wrong': 33,
 'stupid': 34,
 'location': 35,
 'something-else': 36,
 'friends': 37,
 'ask': 38,
 'problem': 39,
 'no-approach': 40,
 'learn-more': 41,
 'user-agree': 42,
 'meditation': 43,
 'user-meditation': 44,
 'pandora-useful': 45,
 'user-advice': 46,
 'learn-mental-health': 47,
 'mental-health-fact': 48,
 'fact-1': 49,
 'fact-2': 50,
 'fact-3': 51,
 'fact-5': 52,
 'fact-6': 53,
 'fact-7': 54,
 'fact-8': 55,
 'fact-9': 56,
 'fact-10': 57,
 'fact-11': 58,
 'fact-12': 59,
 'f

In [23]:
tf.config.run_functions_eagerly(True)
label_encoder = LabelEncoder()
y_train_encoded = df['tags'].replace(tg_map)
y_train_encoded = to_categorical(y_train_encoded, num_classes=80)  # Assuming you have 7 classes
X_train = np.array(df['vect'].tolist())

In [35]:
model.fit(X_train, y_train_encoded, batch_size=512, epochs=500)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - loss: 13.6415
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 8.6012
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 6.4413
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 5.8763
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 5.5433
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 5.1749
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 4.7890
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 4.8855
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 4.5736
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 4.4679
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 4.5383
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 4.3486
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 4.4124
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 4.3273
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 4.3113
Epoch 16/500
1/1 

Epoch 83/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 3.2611
Epoch 84/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 3.2212
Epoch 85/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 3.2601
Epoch 86/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 3.2336
Epoch 87/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 3.0602
Epoch 88/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 3.2364
Epoch 89/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.0907
Epoch 90/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.1816
Epoch 91/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 3.1388
Epoch 92/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 3.1551
Epoch 93/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 3.1076
Epoch 94/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 3.1523
Epoch 95/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.1849
Epoch 96/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 3.1836
Epoch 97/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 3.0831
Epoch 98/5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.5067
Epoch 165/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.5997
Epoch 166/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.6929
Epoch 167/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.5300
Epoch 168/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.6214
Epoch 169/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 2.5677
Epoch 170/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.6246
Epoch 171/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.5839
Epoch 172/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.5434
Epoch 173/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.5940
Epoch 174/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.6215
Epoch 175/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.5501
Epoch 176/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.6176
Epoch 177/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.5472
Epoch 178/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2.3861
Epoch 179

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.1689
Epoch 246/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.1675
Epoch 247/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.1501
Epoch 248/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.1743
Epoch 249/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.1129
Epoch 250/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.1650
Epoch 251/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.1023
Epoch 252/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.1734
Epoch 253/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.1330
Epoch 254/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.0864
Epoch 255/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.0581
Epoch 256/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.1047
Epoch 257/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.0309
Epoch 258/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.0544
Epoch 259/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.0603
Epoch 260

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1.9541
Epoch 327/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.9939
Epoch 328/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9778
Epoch 329/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.8991
Epoch 330/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.0337
Epoch 331/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.9316
Epoch 332/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.9141
Epoch 333/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9710
Epoch 334/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9596
Epoch 335/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9266
Epoch 336/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.9396
Epoch 337/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9817
Epoch 338/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8520
Epoch 339/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.9935
Epoch 340/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.9989
Epoch 341

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8707
Epoch 408/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.9475
Epoch 409/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9425
Epoch 410/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9046
Epoch 411/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.8948
Epoch 412/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9295
Epoch 413/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8552
Epoch 414/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.8594
Epoch 415/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.8926
Epoch 416/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1.8016
Epoch 417/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1.8335
Epoch 418/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.8243
Epoch 419/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.8592
Epoch 420/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8985
Epoch 421/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8564
Epoch 422

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8034
Epoch 489/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8546
Epoch 490/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8327
Epoch 491/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8147
Epoch 492/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.7363
Epoch 493/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8229
Epoch 494/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.7947
Epoch 495/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8134
Epoch 496/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8254
Epoch 497/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8261
Epoch 498/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.8877
Epoch 499/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.8875
Epoch 500/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.8512


In [27]:
word = input('enter: ')
word = nlp(word).vector
word = word.reshape(1, -1)
output = model.predict(word)
predicted_tag_index = np.argmax(output)
predicted_tag = list(tg_map.keys())[list(tg_map.values()).index(predicted_tag_index)]
print(predicted_tag)

enter: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
greeting


In [30]:
def chat_bot_deep_learning():
    run = True
    print("Hello Sharpaxis here how may I help you?\n")
    while run:
        message = input("You: ")
        if message.strip().lower() == 'quit':
            run = False
            break
        word = nlp(message).vector
        word = word.reshape(1, -1)
        output = model.predict(word)
        predicted_tag_index = np.argmax(output)
        predicted_tag = list(tg_map.keys())[list(tg_map.values()).index(predicted_tag_index)]
        bot_response = random.choice(res_dict[predicted_tag])
        print(bot_response)



In [ ]:
tf.data.experimental.enable_debug_mode()
chat_bot_deep_learning()

Hello Sharpaxis here how may I help you?

You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Hi there. How are you feeling today?
You: sleepy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Come Come elucidate your thoughts
You: whta
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Hello there. Glad to see you're back. What's going on in your world right now?
You: im feeling sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
How were you feeling last week?
You: sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Bye! Come back again.
You: im feeling sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Come Come elucidate your thoughts
